In [3]:
# --- Standard and data handling imports ---
# Set path for module imports
import sys
# replace the path below with the absolute path to your `scattering/` folder
pkg_root = "/arc/home/jfaber/baseband_morphologies/chime_dsa_codetections/FLITS/scintillation"
sys.path.insert(0, pkg_root)
# Run this cell to import necessary libraries
import os
import json
import pickle
import logging
import numpy as np
import matplotlib.pyplot as plt

# --- Bokeh Imports for Jupyter (Corrected) ---
from bokeh.io import output_notebook, show
from bokeh.layouts import row, column
from bokeh.models import ColumnDataSource, Slider, Div
from bokeh.plotting import figure
from bokeh.application import Application
from bokeh.application.handlers.function import FunctionHandler

# --- Your Pipeline's Imports ---
# Make sure your scint_analysis package is importable
# (You may need to add its path using sys.path.insert)
try:
    from scint_analysis import config, pipeline, plotting
    from scint_analysis.analysis import lorentzian_model_3_comp
    from scint_analysis.core import ACF
except ImportError as e:
    logging.error(f"Could not import scint_analysis. Make sure it's in your Python path. {e}")

from urllib.parse import urljoin

# --- Imports for the new interactive framework ---
import plotly.graph_objects as go
from ipywidgets import interactive, HBox, VBox, FloatSlider, Label

# --- Your Pipeline's Imports ---
# Make sure your scint_analysis package is importable
try:
    from scint_analysis.analysis import lorentzian_model_3_comp
    from scint_analysis.core import ACF
except ImportError as e:
    logging.error(f"Could not import scint_analysis. Make sure it's in your Python path. {e}")
    

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
# --- Load Pre-calculated ACF Data ---
SUBBAND_INDEX = 1 # Choose which sub-band to analyze (0, 1, 2...)
ACF_RESULTS_PATH = '/arc/home/jfaber/baseband_morphologies/chime_dsa_codetections/FLITS/scintillation/data/cache/casey/casey_acf_results.pkl' # Adjust path if needed

#try:
with open(ACF_RESULTS_PATH, 'rb') as f:
    acf_results = pickle.load(f)
logging.info(f"Successfully loaded ACF results from {ACF_RESULTS_PATH}")

# Create a clean ACF object for the selected sub-band
lags = acf_results['subband_lags_mhz'][SUBBAND_INDEX]
data = acf_results['subband_acfs'][SUBBAND_INDEX]
#errors = acf_results.get('subband_acfs_err', [np.ones_like(data)])[SUBBAND_INDEX]

# Ensure errors is a valid array for calculations
if errors is None:
    errors = np.ones_like(data)

acf_obj = ACF(acf_data=data, lags_mhz=lags, acf_err=None)

#except FileNotFoundError:
#    logging.error(f"ERROR: ACF results file not found at {ACF_RESULTS_PATH}. Please run the main pipeline first.")
#except Exception as e:
#    logging.error(f"An error occurred loading or processing the ACF data: {e}")

In [21]:
print([i for i in acf_results])

['subband_acfs', 'subband_lags_mhz', 'subband_center_freqs_mhz', 'subband_channel_widths_mhz', 'subband_num_channels', 'noise_template', 'sigma_self_mhz']


In [7]:





def hub_proxy_url(port: int) -> str:
    """
    Return the proxied URL 'https://<hub>/user/<you>/proxy/<port>/'
    understood by JupyterHub and Binder.
    """
    base = os.getenv("JUPYTERHUB_SERVICE_PREFIX", "/")
    # ensure trailing slash then append 'proxy/<port>/'
    return urljoin(base, f"proxy/{port}/")

# Run this cell to load the data for one sub-band
SUBBAND_INDEX = 0 # Choose which sub-band to analyze (0, 1, 2...)
ACF_RESULTS_PATH = '/arc/home/jfaber/baseband_morphologies/chime_dsa_codetections/FLITS/scintillation/data/cache/casey/casey_acf_results.pkl' # Adjust path if needed

try:
    with open(ACF_RESULTS_PATH, 'rb') as f:
        acf_results = pickle.load(f)
    logging.info(f"Successfully loaded ACF results from {ACF_RESULTS_PATH}")
except FileNotFoundError:
    logging.error(f"ERROR: ACF results file not found at {ACF_RESULTS_PATH}. Please run the main pipeline first.")
    
# Create a clean ACF object for the selected sub-band
lags = acf_results['subband_lags_mhz'][SUBBAND_INDEX]
data = acf_results['subband_acfs'][SUBBAND_INDEX]
# Use the robustly calculated errors for chi-squared calculation
errors = np.sqrt(acf_results['subband_acfs_err'][SUBBAND_INDEX]**2) if 'subband_acfs_err' in acf_results else np.ones_like(data)

acf_obj = ACF(acf_data=data, lags_mhz=lags, acf_err=errors)

# Run this cell to define the application logic
def make_document(doc):
    """
    This function is a self-contained Bokeh application.
    It takes a Bokeh document `doc` and adds the interactive plot to it.
    """
    # --- Define Model and Parameters for Sliders ---
    model_func = lorentzian_model_3_comp
    param_names = ['gamma1', 'm1', 'gamma2', 'm2', 'gamma3', 'm3', 'c3']
    
    # Use initial guesses from your pipeline config as the starting point
    p0 =           [0.05,  0.5,  0.2,   0.4,  0.8,   0.3,  0.0] 
    lower_bounds = [1e-4,  0,    1e-4,  0,    1e-4,  0,    -0.2]
    upper_bounds = [1.0,   1.5,  2.0,   1.5,  5.0,   1.5,  0.2]

    # --- Prepare Data Sources ---
    x_lags = acf_obj.lags
    y_data = acf_obj.acf
    y_model_init = model_func(x_lags, *p0)

    source_data = ColumnDataSource(data=dict(x=x_lags, y=y_data))
    source_model = ColumnDataSource(data=dict(x=x_lags, y=y_model_init))
    source_resid = ColumnDataSource(data=dict(x=x_lags, y=(y_data - y_model_init)))

    # --- Set up Plots ---
    plot = figure(height=400, width=700, title=f"Interactive ACF Fit (Sub-band {SUBBAND_INDEX})", x_axis_label="Frequency Lag (MHz)")
    plot.circle('x', 'y', source=source_data, legend_label="ACF Data", color="navy", alpha=0.6)
    plot.line('x', 'y', source=source_model, legend_label="Interactive Model", color="crimson", line_width=2)
    plot.legend.location = "top_right"

    plot_residual = figure(height=200, width=700, title="Residuals", x_range=plot.x_range, x_axis_label="Frequency Lag (MHz)")
    plot_residual.line('x', 'y', source=source_resid, line_color="black")
    plot_residual.line(x_lags, np.zeros_like(x_lags), line_dash='dashed', color='gray')

    # --- Set up Widgets ---
    sliders = [Slider(title=name, value=val, start=low, end=high, step=(high-low)/200)
               for name, val, low, high in zip(param_names, p0, lower_bounds, upper_bounds)]
    gof_div = Div(text="Reduced Chi-Squared: N/A", width=300, style={'font-size': '1.1em', 'font-weight': 'bold'})

    # --- Define the Callback Function ---
    def update_fit(attr, old, new):
        # 1. Current slider values
        p = [s.value for s in sliders]

        # 2. Model prediction
        y_model = model_func(x_lags, *p)

        # 3. Update the model line (replace the whole dict -> triggers events)
        source_model.data = dict(source_model.data, y=y_model)

        # 4. Residuals
        resid = y_data - y_model
        source_resid.data = dict(source_resid.data, y=resid)

        # 5. Goodness-of-fit read-out
        err = np.maximum(acf_obj.err, 1e-9)        # keep σ > 0
        dof = len(y_data) - len(p)
        if dof > 0:
            redchi = np.sum((resid / err)**2) / dof
            gof_div.text = f"<b>Reduced χ²: {redchi:.3f}</b>"

    for w in sliders:
        w.on_change('value', update_fit)
        
    # Trigger the initial GoF calculation
    update_fit(None, None, None)

    # --- Assemble Layout and Add to Document ---
    inputs = column(sliders)
    plots = column(plot, plot_residual)
    layout = row(inputs, plots, gof_div)
    
    doc.add_root(layout)
    doc.title = "ACF Explorer"

ModuleNotFoundError: No module named 'panel'

In [4]:
# This tells Bokeh to generate output in the notebook
output_notebook()

# This runs make_document as an application and embeds it
app = Application(FunctionHandler(make_document))
show(app, notebook_url=hub_proxy_url)

Loading BokehJS ...

In [5]:

# This runs make_document as an application and embeds it
app = Application(FunctionHandler(make_document))


In [6]:
show(app, notebook_url=hub_proxy_url)